<a href="https://colab.research.google.com/github/adam-bozman/tutorials/blob/main/FuzzyPermnoMatching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install libraries
!pip install wrds -q
!pip install fuzzywuzzy -q
!pip install python-Levenshtein -q
!pip install tqdm -q

In [ ]:
import pandas as pd
import wrds
from fuzzywuzzy import process
from tqdm import tqdm

# Enable progress_apply in pandas with tqdm
tqdm.pandas()

In [ ]:
# Connect to WRDS
conn = wrds.Connection()

Enter your WRDS username [root]:bozmanadam
Enter your password:··········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: y
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [ ]:
# Load the Excel file
df = pd.read_excel('data.xlsx')  # change to pd.read_csv('your_file.csv') for CSV files

In [ ]:
# Fetch companies with consideration to the end of 2022 and early 2023
refined_query = """
    SELECT DISTINCT permno, ticker, comnam
    FROM crsp.msenames
    WHERE (namedt <= '2023-01-01' AND (nameendt >= '2023-01-01' OR nameendt IS NULL OR nameendt = '2022-12-30'))
    OR (namedt BETWEEN '2023-01-01' AND '2023-01-31');
"""
all_companies = conn.raw_sql(refined_query)

In [ ]:
# Check the result
print(all_companies.head())

    permno ticker                            comnam
0  89216.0    AYI                 ACUITY BRANDS INC
1  75837.0    DSM  B N Y MELLON STRAT MUNI BOND FUN
2  22688.0   VHNA      VAHANNA TECH EDGE ACQ I CORP
3  12303.0   IVOG        VANGUARD ADMIRAL FUNDS INC
4  19192.0   HUGE                    FSD PHARMA INC


In [ ]:
popular_brands = {
    "apple": "Apple Inc.",
    "google": "Alphabet Inc.",
    "microsoft": "Microsoft Corp.",
    "amazon": "Amazon.com Inc.",
    "facebook": "Facebook Inc.",  # Note: Facebook rebranded as "Meta" in late 2021
    "coca-cola": "The Coca-Cola Co",
    "samsung": "Samsung Electronics Co Ltd",
    "disney": "Walt Disney Co",
    "mcdonald's": "McDonald's Corp",
    "toyota": "Toyota Motor Corp",
    "mercedes-benz": "Daimler AG",
    "nike": "NIKE Inc",
    "intel": "Intel Corp",
    "louis vuitton": "LVMH Moet Hennessy Louis Vuitton SE",
    "walmart": "Walmart Inc.",
    "bmw": "Bayerische Motoren Werke AG",
    "pepsi": "PepsiCo Inc.",
    "adidas": "Adidas AG",
    "starbucks": "Starbucks Corp.",
    "netflix": "Netflix Inc.",
    "american express": "American Express Co",
    "visa": "Visa Inc.",
    "ikea": "Inter IKEA Systems B.V.",  # Note: IKEA might not match stock data as it's privately held
    "oracle": "Oracle Corp",
    "nintendo": "Nintendo Co Ltd",
    "ford": "Ford Motor Co",
    "honda": "Honda Motor Co Ltd",
    "ge": "General Electric Co",
    "h&m": "H & M Hennes & Mauritz AB",
    "gucci": "Kering SA",  # Gucci is owned by Kering
    "hermes": "Hermes International SCA",
    "cisco": "Cisco Systems Inc",
    "porsche": "Porsche AG",
    "marlboro": "Philip Morris International Inc.",
    "chanel": "Chanel International B.V.",  # Note: Might not match stock data as Chanel is privately held
    "accenture": "Accenture Plc",
    "audi": "Audi AG",
    "nissan": "Nissan Motor Co Ltd",
    "zara": "Inditex SA",
    "siemens": "Siemens AG",
    "mastercard": "Mastercard Inc",
    "hp": "Hewlett-Packard Co",
    "lego": "The LEGO Group",  # Note: LEGO is privately held, might not match stock data
    "ups": "United Parcel Service Inc.",
    "gillette": "Procter & Gamble Co",
    "goldman sachs": "Goldman Sachs Group Inc",
    "puma": "Puma SE",
    "kellogg's": "Kellogg Co",
    "heinz": "The Kraft Heinz Co",
    "sony": "Sony Corp",
    "boeing": "The Boeing Co",
    "shell": "Royal Dutch Shell PLC",
    "colgate": "Colgate-Palmolive Co",
    "j.p. morgan": "JPMorgan Chase & Co",
    "rolex": "Rolex SA",  # Note: Rolex is privately held, might not match stock data
    "chevrolet": "General Motors Co",
    "dell": "Dell Inc.",
    "nestle": "Nestle SA",
    "paypal": "PayPal Holdings Inc",
    "att": "AT&T Inc.",
    "cartier": "Compagnie Financiere Richemont SA",  # Cartier is owned by Richemont
    "morgan stanley": "Morgan Stanley",
    "ibm": "International Business Machines Corp",
    "danone": "Danone SA",
    "fedex": "FedEx Corp",
    "ferrari": "Ferrari NV",
    "volkswagen": "Volkswagen AG",
    "kfc": "Yum! Brands Inc.",  # KFC is owned by Yum! Brands
    "hyundai": "Hyundai Motor Co",
    "spotify": "Spotify Technology SA",
    "airbnb": "Airbnb Inc",
    "zoom": "Zoom Video Communications Inc.",
    "tesla": "Tesla Inc",
    "dior": "Christian Dior SE",
    "prada": "Prada Spa",
    "adobe": "Adobe Systems Inc",
    "twitter": "Twitter Inc",
    "lenovo": "Lenovo Group Ltd",
    "uber": "Uber Technologies Inc",
    "chrysler": "Stellantis NV",  # As of my last update, Chrysler is part of Stellantis
    "harley-davidson": "Harley-Davidson Inc",
    "jack daniel's": "Brown-Forman Corp",
    "heineken": "Heineken NV",
    "sprite": "The Coca-Cola Co",  # Sprite is owned by Coca-Cola
    "reebok": "Adidas AG"  # Reebok is owned by Adidas
    # ... and more. Adjust as needed!
}

In [ ]:
from fuzzywuzzy import fuzz

def get_best_match(brand_name):
    # Try to find the brand in the popular_brands dictionary with a threshold of 90%
    matching_brands = {k: v for k, v in popular_brands.items() if fuzz.ratio(brand_name.lower(), k) >= 80}

    # If we found a match in popular brands
    if matching_brands:
        best_match_brand = list(matching_brands.values())[0]  # Get the first match
        matched_rows = all_companies[all_companies['comnam'].str.contains(best_match_brand, case=False)]

        if not matched_rows.empty:
            all_permnos = matched_rows['permno'].unique()
            return pd.Series([best_match_brand, ', '.join(map(str, all_permnos)), matched_rows['ticker'].iloc[0]])

    # If no match in popular brands, perform the fuzzy match on all_companies
    else:
        best_match = process.extractOne(brand_name, all_companies['comnam'], score_cutoff=80)

        if best_match:
            best_comnam, score = best_match[:2]
            matched_rows = all_companies[all_companies['comnam'].str.contains(best_comnam, case=False)]

            # Collecting all permnos for the matched company
            if not matched_rows.empty:
                all_permnos = matched_rows['permno'].unique()
                return pd.Series([best_comnam, ', '.join(map(str, all_permnos)), matched_rows['ticker'].iloc[0]])

    # Return 'none' if no match found at all
    return pd.Series(["", "", ""])

In [ ]:
df[['matched_comnam', 'matched_permnos', 'matched_ticker']] = df['Brand'].progress_apply(get_best_match)

100%|██████████| 391/391 [03:44<00:00,  1.74it/s]


In [ ]:
df.to_excel('output_with_matches.xlsx', index=False)